# exp012
[Notion](https://www.notion.so/exp012-1ed9dc9b29ff41cdbb9493131c83ece4?pvs=4)  
exp004,009,011によって作成した深層学習モデルにより、コンペ提出用のパイプラインを構築する.

In [1]:
import os
import random
import sys
import warnings
warnings.filterwarnings('ignore')
from collections import defaultdict
from typing import Tuple, Any, Dict

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

# リポジトリtopに移動
while os.path.basename(os.getcwd()) != 'rsna-2023':
    os.chdir('../')
    if os.getcwd() == '/':
        raise Exception('Could not find project root directory.')
    
from src.data_io import load_dicom_series
from src.segmentation.dataset import TestDataset as SegTestDataset
from src.segmentation.model import load_models as seg_load_models
from src.segmentation.trainer import inference as seg_inference
from src.classification.dataset import TestDatasetBowelExtra, TestDatasetSolidOrgans
from src.image_processing import apply_preprocess, crop_organ, kidney_split, resize_volume, apply_postprocess, kidney_specific, resize_3d, resize_1d
from src.classification.model import load_models as cls_load_models
from src.classification.trainer import inference as cls_inference
from src.metrics import logloss

# Configs

In [2]:
class CFG_INF:
    exp_name = 'exp_012'
    # evaluation時：'train', submission時：'test'
    phase = 'train'
    base_dir = 'data/rsna-2023-abdominal-trauma-detection'
    image_dir = f'data/rsna-2023-abdominal-trauma-detection/{phase}_images'
    # dataframeはこのconfigにもたせ、phaseで対応できるようにする.
    if phase == 'train':
        df = pd.read_csv(os.path.join(base_dir, 'train.csv'))
    elif phase == 'test':
        df = pd.read_csv(os.path.join(base_dir, 'sample_submission.csv'))
    df_series_meta = pd.read_csv(os.path.join(base_dir, f'{phase}_series_meta.csv'))
    image_size = (512, 512)
    # sample submissionで極端にスライス数が少ない場合があるため対応.
    min_slices = 10
    # 推論時間制限のため
    max_slices = 500
    max_series = 1

class CFG_SEG:
    exp_name = 'exp_004'
    # model config
    backbone = 'efficientnet-b3'
    n_ch = 1
    n_class = 4 # 学習時は腎臓の左右を区別しないので、5->4
    # hyper params
    init_lr = 1e-3
    min_lr = 1e-6
    weight_decay = 1e-4
    image_size = (512, 512)
    batch_size = 32
    amp = True
    n_epoch = 20
    iteration_per_epoch = 200
    pretrain = True
    freeze_epochs = 1
    noaug_epochs = 1
    # fold config
    n_fold = 6
    include_evaluation = False
    train_folds = 1
    # path
    image_dir = "data/dataset001/train_images"
    mask_dir = "data/dataset001/segmentations"
    model_save_dir = "outputs"
    # other config
    seed = 42
    num_workers = 0
    num_gpus = 2
    progress_bar = True
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class CFG_LSK:
    exp_name = 'exp_009'
    # model config
    # timm backbone
    backbone = 'efficientnet-b1'
    n_ch = 1
    expand_ch_dim = True
    # n_class: healthy, low, high
    n_class = 3
    # hyper params
    init_lr = 1e-4
    min_lr = 1e-6
    weight_decay = 1e-4
    image_size = (128, 128, 128)
    batch_size = 64
    amp = True
    n_epoch = 20
    pretrain = False
    freeze_epochs = 0
    noaug_epochs = 1
    # fold config
    n_fold = 6
    include_evaluation = False
    train_folds = 1
    # path
    image_dir = "data/dataset002"
    model_save_dir = "outputs"
    # other config
    seed = 42
    num_workers = 0
    num_gpus = 2
    progress_bar = True
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class CFG_BE:
    exp_name = 'exp_011'
    # model config
    # timm backbone
    backbone = 'efficientnet_b4'
    # n_ch: z軸方向のスライス数
    n_ch = 1 # support only 1
    expand_ch_dim = False
    # n_class: bowel_injury, extravasation
    n_class = 2
    label_smoothing = None #Optional(float)
    # hyper params
    init_lr = 5e-5
    min_lr = 1e-6
    weight_decay = 1e-4
    image_size = (512, 512)
    batch_size = 64
    amp = True
    n_epoch = 20
    iteration_per_epoch = 100
    pretrain = True
    freeze_epochs = 1
    noaug_epochs = 1
    # fold config
    n_fold = 6
    include_evaluation = False
    train_folds = 1
    # path
    image_dir = "data/dataset001"
    model_save_dir = "outputs"
    # other config
    seed = 42
    num_workers = 0
    num_gpus = 2
    progress_bar = True
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# organs dict (for SEG and LSK models)
organ_index_dict_inv = {
    0: 'liver',
    1: 'spleen',
    2: 'kidney',
    3: 'bowel'
}
organ_index_dict = {v: k for k, v in organ_index_dict_inv.items()}

# labels dict (for BE models)
label_index_dict_inv = {
    0: 'bowel',
    1: 'extravasation'
}

In [4]:
class Inference:
    """推論パイプライン."""
    def __init__(self,CFG_INF: Any, CFG_SEG: Any, CFG_LSK: Any, CFG_BE: Any):
        self.CFG_INF = CFG_INF
        self.CFG_SEG = CFG_SEG
        self.CFG_LSK = CFG_LSK
        self.CFG_BE = CFG_BE

        self.seg_models = seg_load_models(CFG_SEG)
        self.lsk_models = cls_load_models(CFG_LSK)
        self.be_models = cls_load_models(CFG_BE, framework="timm")
    
    def __call__(self, pid: int) -> tuple:
        """inference process.
        1. load images from dicom files.
        2. create segmentation masks.
        3. create liver, spleen, kidney volumes.
        4. inference lsk models.
        5. inference be models.
        Args:
            pid (int): patient id.
        Return example:
            dict: {
            'pid': 0,
            'bowel_healthy': 0.0,
            'bowel_injury': 0.0,
            'extravasation_healthy': 0.0,
            'extravasation_injury': 0.0,
            'kidney_healthy': 0.0,
            'kidney_low': 0.0,
            'kidney_high': 0.0,
            'liver_healthy': 0.0,
            'liver_low': 0.0,
            'liver_high': 0.0,
            'spleen_healthy': 0.0,
            'spleen_low': 0.0,
            'spleen_high': 0.0
            }
        Note:
            - １症例に複数シリーズ存在する場合、各シリーズに対して推論を行い、全予測結果の最大値を採用する.
            - 推論時間的に厳しければ、最初のシリーズのみを採用するなど検討.
        """
        df_stydy = self.CFG_INF.df_series_meta[self.CFG_INF.df_series_meta['patient_id']==pid]
        preds = defaultdict(list)
        for sid in df_stydy['series_id'].to_list()[:self.CFG_INF.max_series]:
            data = self.load_data(pid, sid)
            if data is None:
                continue
            lsk_preds = self.lsk_prediction(data)
            be_preds = self.be_prediction(data)
            
            for idx, organ in organ_index_dict_inv.items():
                if idx == 3:
                    continue
                preds[organ].append(lsk_preds[idx])
            for idx, label in label_index_dict_inv.items():
                pred = np.array([be_preds[idx]])
                preds[label].append(pred)

        ret = {'patient_id': pid}
        for k,v in preds.items():
            v = np.array(v)
            ret[k] = np.max(v, axis=0)
        ret = self.convert_submission_format(ret)
        return ret

    def load_data(self, pid: int, sid: int)-> np.ndarray:
        """dicomから画像を読み込む.
        Args:
            pid (int): patient id.
            sid (int): series id.
        Returns:
            np.ndarray: (Z, H, W) normalized CT series.
        Note:
            - preprocessは全モデル共通なので、ここで行う.
            - H, Wはすべてself.CFG_INF.image_sizeにresizeされる.
        """
        series_path = os.path.join(self.CFG_INF.image_dir, str(pid), str(sid))
        # sample submissionでこういう例が存在する.
        if not os.path.exists(series_path): 
            return None
        image_arr, path_list, meta_list = load_dicom_series(series_path, self.CFG_INF.max_slices)
        image_arr = apply_preprocess(image_arr, resize=self.CFG_INF.image_size)
        # sample submission対応
        if len(image_arr) < self.CFG_INF.min_slices:
            image_arr = resize_1d(image_arr, self.CFG_INF.min_slices, axis=0)
        return image_arr
    
    def lsk_prediction(self, data: np.ndarray)-> np.ndarray:
        """liver, spleen, kidneyの予測値を返す.
        Args:
            data: (Z, H, W).
        Returns:
            np.ndarray: (organs, grades).
        """
        volumes = self.get_lsk_volumes(data) # (organs, z, h, w)
        lsk_iterator = self.pseudo_iterator(self.CFG_LSK, volumes)
        pred = cls_inference(self.CFG_LSK, self.lsk_models, lsk_iterator)
        return pred

    def get_lsk_volumes(self, data: np.ndarray)->Dict[str, np.ndarray]:
        """Segmentationからliver, spleen, kidneyのvolume dataを作成.
        Args:
            data: (Z, H, W).
        Returns:
            np.ndarray: (organs, z, h, w).
        Note:
            - organsはliver, spleen, kidneyの順番.
            - この関数内でCFG.LSK.image_sizeのreshapeまで行う.
            - 腎臓は左右を分離してからくっつけ直すという特殊な処理が必要.
        """
        masks = self.get_segmentation(data)
        masks = apply_postprocess(self.CFG_SEG, masks)
        arr = []
        for idx, organ in organ_index_dict_inv.items():
            if idx == 3:
                continue
            organ_segment = masks[..., idx]
            if organ_segment.sum() == 0:
                arr.append(np.zeros(self.CFG_LSK.image_size))
                continue
            img_cropped, mask_cropped = crop_organ(data, organ_segment)
            if organ == "kidney":
                kidney_r, kidney_l = kidney_split(img_cropped, mask_cropped)
                img_cropped = kidney_specific(self.CFG_LSK, kidney_r, kidney_l)
            else:
                img_cropped = resize_3d(img_cropped, self.CFG_LSK.image_size)
            arr.append(img_cropped)
        arr = np.stack(arr, axis=0)
        return arr
    
    def get_segmentation(self, data: np.ndarray)->np.ndarray:
        """Segmentation modelを使って、各臓器のマスクを作成.
        Args:
            data: (Z, H, W).
        Returns:
            mask: (z, h, w, ch) binarized."""
        seg_iterator = self.pseudo_iterator(self.CFG_SEG, data)
        pred = seg_inference(self.CFG_SEG, self.seg_models, seg_iterator)
        pred = (pred > 0.5).astype(np.uint8)
        return pred
    
    def be_prediction(self, data: np.ndarray)-> np.ndarray:
        """bowel_injury及びextravasation_injuryの予測を行う.
        Args:
            data: (Z, H, W).
        Returns:
            np.ndarray: [bowel_injury_pred, extravasation_injury_pred].
            example: [0.1, 0.9].
        """
        be_iterator = self.pseudo_iterator(self.CFG_BE, data)
        pred = cls_inference(self.CFG_BE, self.be_models, be_iterator)
        pred = self.be_prediction_postprocess(pred)
        return pred
    
    def be_prediction_postprocess(self, pred: np.ndarray)-> np.ndarray:
        """スライスごとの予測をシリーズの予測に変換する.
        Args:
            pred: (len(data),['bowel_injury', 'extravasation_injury']).
        Returns:
            np.ndarray: ['bowel_injury', 'extravasation_injury'].
        Note:
            - 予測値の最大値から外れ値を考慮した2%percentileを採用する.
        """
        bowel = pred[:, 0]
        extravasation = pred[:, 1]
        bowel = np.percentile(bowel, 98)
        extravasation = np.percentile(extravasation, 98)
        return np.array([bowel, extravasation])

    def pseudo_iterator(self, CFG: Any, images: np.ndarray)-> tuple:
        """evaluation iterator.
        Args:
            CFG: config.
            images: (batch dim, H, W) or (batch dim, Z, H, W).
        """
        batch = CFG.batch_size
        for i in range(0, len(images), batch):
            arr = images[i : i + batch]
            arr = self.add_ch_dim(arr)
            arr = torch.from_numpy(arr.astype(arr.dtype, copy=False))
            yield arr

    def add_ch_dim(self, images: np.ndarray)-> np.ndarray:
        """1次元目にchannel dimを追加する."""
        return images[:, np.newaxis, ...]

    def convert_submission_format(self, pred: dict)->dict:
        """提出形式に変換する."""
        converted = dict()
        for idx, organ in organ_index_dict_inv.items():
            if idx == 3:
                continue
            for idx, grade in enumerate(['healthy', 'low', 'high']):
                converted[f'{organ}_{grade}'] = pred[organ][idx]
        for idx, label in label_index_dict_inv.items():
            converted[f'{label}_healthy'] = 1 - pred[label][0]
            converted[f'{label}_injury'] = pred[label][0]

        converted['patient_id'] = pred['patient_id']
        return converted

In [5]:
inference_instance = Inference(CFG_INF, CFG_SEG, CFG_LSK, CFG_BE)

In [6]:
ct = 0
results = []
for pid in tqdm(CFG_INF.df['patient_id'].to_list()):
    result = inference_instance(pid)
    results.append(result)
    ct += 1
    if ct == 10**8:
        break

  0%|          | 12/3147 [01:36<12:57:07, 14.87s/it]

In [ ]:
%debug

> /home/medphys3/anaconda3/envs/env_1/lib/python3.9/site-packages/numpy/core/shape_base.py(422)stack()
    420     arrays = [asanyarray(arr) for arr in arrays]
    421     if not arrays:
--> 422         raise ValueError('need at least one array to stack')
    423 
    424     shapes = {arr.shape for arr in arrays}

> <__array_function__ internals>(180)stack()

> /home/medphys3/competition/rsna-2023/src/image_processing.py(114)apply_preprocess()
    112         for i in range(image.shape[0]):
    113             new_arr.append(cv2.resize(image[i], (resize[1], resize[0])))
--> 114         return np.stack(new_arr)
    115 
    116     return image

*** NameError: name 'ls' is not defined
> /tmp/ipykernel_13065/4003132833.py(83)load_data()
     81             return None
     82         image_arr, path_list, meta_list = load_dicom_series(series_path, self.CFG_INF.max_slices)
---> 83         image_arr = apply_preprocess(image_arr, resize=self.CFG_INF.image_size)
     84         # sample sub

In [ ]:
submission = pd.DataFrame(results)
order = CFG_INF.df.columns.tolist()
if "any_injury" in order:
    order.remove("any_injury")
submission = submission[order]

In [ ]:
submission

,patient_id,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,liver_high,spleen_healthy,spleen_low,spleen_high
0,48843,0.928918,0.071082,0.939963,0.060037,0.955287,0.037823,0.011887,0.865929,0.098345,0.045093,0.555941,0.203244,0.279940
1,50046,0.999978,0.000022,0.998273,0.001727,1.000000,0.000000,0.000000,0.741396,0.178594,0.089132,0.824086,0.149163,0.044591
2,63706,0.999982,0.000018,0.996831,0.003169,1.000000,0.000000,0.000000,0.674272,0.186021,0.151613,0.710415,0.170997,0.133056
